In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from datetime import datetime
import random

import utils.utils as utils
import evaluation.inception_network as inception_network
import evaluation.metrics.maximum_mean_discrepancy as maximum_mean_discrepancy
import evaluation.metrics.mmd as mmd_function
import data.preprocessing as preprocessing
import data.audio_transforms as audio_transforms
import data.loaders as loaders

In [ ]:
device = 'cpu'

In [ ]:
# configuration dictionary
config = {
    "model_name": "footsteps_inception_model_best_2021-09-26.pt",
    "comments": "inception trained on footsteps dataset",
    # "state_dict_path": "/homes/mc309/hifi-wavegan/drumgan_evaluation/evaluation/inception_models/footsteps_inception_model_best_2021-09-26.pt",
    "state_dict_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/hifi-wavegan/drumgan_evaluation/evaluation/inception_models/footsteps_inception_model_best_2021-09-26.pt",
    
    # real samples used to train inception model
    # "real_samples_path": "/homes/mc309/_data/zapsplat_misc_shoes_misc_surfaces_inception_network/",
    # real samples used to train gan
    # "real_samples_path": "/homes/mc309/_data/zapsplat_pack_footsteps_high_heels_1s_aligned_for_inception_score/",
    # gan synthesised samples
    # "synth_samples_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/hifi-wavegan/checkpoints/2021-09-20_13h23m-hifi/120k_generated_audio_large_for_is_and_kid/",
    # "synth_samples_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/hifi-wavegan/checkpoints/2021-09-20_19h46m-wave/120k_generated_audio_large_for_is_and_kid/",
    
    "output_path": "evaluation",
    "output_folder": "evaluation_metrics",
    
    "batch_size": 20,

    "real_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/hifi-wavegan/checkpoints/2021-09-20_13h23m-hifi/120k_generated_audio_large_for_is_and_kid/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },

    "synth_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "/Users/Marco/Documents/OneDrive - Queen Mary, University of London/PHD/REPOS/hifi-wavegan/checkpoints/2021-09-20_19h46m-wave/120k_generated_audio_large_for_is_and_kid/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },
    
    "transform_config": {
        "transform": "stft",
        "fade_out": True,
        "fft_size": 1024,
        "win_size": 1024,
        "n_frames": 64,
        "hop_size": 256,
        "log": False,
        "ifreq": False,
        "sample_rate": 16000,
        "audio_length": 8192
    }
}

In [ ]:
model_name = config['model_name']
state_dict_path = config['state_dict_path']
output_path = utils.mkdir_in_path(config['output_path'], config['output_folder'])

In [ ]:
# setup dataloader and processor for real and synthesised samples
real_samples_loader_config = config['real_samples_loader_config']
synth_samples_loader_config = config['synth_samples_loader_config']

transform_config = config['transform_config']
transform = transform_config['transform']

dbname = real_samples_loader_config['dbname']

batch_size = config['batch_size']

processor = preprocessing.AudioProcessor(**transform_config)

loader_module = loaders.get_data_loader(dbname)

real_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **real_samples_loader_config)
synth_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **synth_samples_loader_config)

n_real_samples = len(real_samples_loader)
print('n_real_samples: ', n_real_samples)
n_synth_samples = len(synth_samples_loader)
print('n_synth_samples: ', n_synth_samples)

real_samples_data_loader = DataLoader(real_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)
synth_samples_data_loader = DataLoader(synth_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

# load inception model
device = 'cuda' if utils.GPU_is_available() else 'cpu'

state_dict = torch.load(state_dict_path, map_location=device)
inception_footsteps = inception_network.SpectrogramInception3(state_dict['fc.weight'].shape[0], aux_logits=False)
inception_footsteps.load_state_dict(state_dict)
# inception_footsteps = inception_footsteps.to(device)

mel = audio_transforms.MelScale(sample_rate=transform_config['sample_rate'],
                fft_size=transform_config['fft_size'],
                n_mel=transform_config.get('n_mel', 256),
                rm_dc=True)
# mel = mel.to(device)

In [ ]:
# compute embeddings for real samples
real_logits = []

with torch.no_grad():
    for batch_idx, data in enumerate(real_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        real_logits.append(preds)

        print('batch: ', batch_idx)
    
real_logits = torch.cat(real_logits, dim=0)

In [ ]:
# compute embeddings for synthesised samples
synth_logits = []

with torch.no_grad():
    for batch_idx, data in enumerate(synth_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        synth_logits.append(preds)

        print('batch: ', batch_idx)
    
synth_logits = torch.cat(synth_logits, dim=0)

In [ ]:
# compute KID squared distance
mmd_distance = maximum_mean_discrepancy.mmd(real_logits, synth_logits)
print('kid squared mmd: ', float(mmd_distance))

In [ ]:
# MMD A vs B
n_mmds = 1000
n_samples_A = real_logits.shape[0]
n_samples_B = synth_logits.shape[0]
n_samples_mmd = int(min(n_samples_A, n_samples_B) / 2)

mmds = []

for i in range(0, n_mmds):
    print(i, end = ' ')
    idx1 = sorted(random.sample(range(0, n_samples_A-1), n_samples_mmd))
    idx2 = sorted(random.sample(range(0, n_samples_B-1), n_samples_mmd))
    real_logits_subset = real_logits[idx1]
    synth_logits_subset = synth_logits[idx2]
    mmd = mmd_function.mmd(x=real_logits_subset, y=synth_logits_subset, distance='manhattan')
    mmds.append(mmd)
print()
print('kid mean l1 mmd: ',np.average(mmds))
print('kid std l1 mmd: ', np.std(mmds))

In [ ]:
# save results
output_file = f'{output_path}/KID_{model_name}_{datetime.now().strftime("%d-%m-%y_%H_%M")}.txt'

with open(output_file, 'w') as f:
    f.write(str(mean_MMD)+'\n')
    f.write(str(var_MMD))
    f.close()